# Survery Data Analysis

We use pandas to present some characteristics of our data graphically!

In [ ]:
# My exact environment is available in the directory
# however, it's really bloated and I suggest you
# manually install the necessary packages

%matplotlib inline

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('questionario_post_pulizia.csv')

In [ ]:
df["usa_mezzi"] = df["usa_mezzi"].replace({"Si": True, "No": False})

Ecco una rapida occhiata al Dataframe:

In [ ]:
# Il df è di questa forma
# Index(['orario', 'eta', 'dispositivo', 'usa_mezzi', 'freq_mezzi',
#       'provincia_mezzi', 'che_mezzi', 'disservizi', 'applicazioni',
#       'affidabilita_app', 'interesse_crowdsource', 'interesse_partecipazione',
#       'interesse_posizione', 'preferenza_salita', 'interesse_classifica',
#       'interesse_sblocco_feature', 'interesse_sblocco_custom',
#       'interesse_donazioni', 'suggerimenti', 'Treno', 'Bus', 'Tram', 'Metro',
#       'province'],
#      dtype='object')

In [ ]:
cols = ["orario", "eta", "dispositivo", "applicazioni", "provincia_mezzi", "disservizi"]

# display head

df[cols].head()

Per prima cosa, guardiamo rapidamente che dispositivi sono stati utilizzati:

In [ ]:
#group by dispositivo (Android, iOS)

devices = df.groupby('dispositivo')

import seaborn as sns
import matplotlib.pyplot as plt

#paper style
sns.set_theme(style="whitegrid")
sns.set_context("paper")

#plot

fig, ax = plt.subplots(figsize=(10, 6))

sns.countplot(x="dispositivo", data=df, ax=ax)

ax.set_title("Dispositivo utilizzato per compilare il questionario")

ax.set_xlabel("Dispositivo")

ax.set_ylabel("Numero di risposte")

plt.show()


Diamo anche un occhiata alle provincie:

In [ ]:
# plot answers by province

fig, ax = plt.subplots(figsize=(10, 5))

sns.countplot(x="provincia_mezzi", data=df, ax=ax,palette="summer")

ax.set_xticklabels(rotation=30, labels=ax.get_xticklabels())

ax.set(xlabel='Provincia', ylabel='Numero di risposte')

fig.suptitle('Utilizzo dei mezzi pubblici per provincia')

plt.savefig('provincia_mezzi.png', dpi=300, bbox_inches='tight')




Ci accorgiamo che la maggior parte delle persone proviene da Roma (no surprise), una minoranza delle risposte
arriva anche da Napoli e Milano

Per proseguire, diamo uno sguardo anche alle risposte in ordine di età.

In [ ]:
# Number of answers by age group
g = sns.countplot(x="eta", data=df, palette="summer", order=["<18", "18-25", "25-35", "35-55", "55>"])

g.set(xlabel='Età', ylabel='Numero di risposte')
g.set_xticklabels(rotation=30, labels=g.get_xticklabels())

fig.suptitle('Numero di risposte per età')

plt.savefig('eta.png', dpi=300, bbox_inches='tight')


Adesso guardiamo i mezzi di trasporto preferiti, e, per curiosità, incrociamoli
con i dispositivi utilizzati

In [ ]:
mezzi = ["Treno", "Bus", "Tram", "Metro"]

# split the "che_mezzi" column and create a new column for each mean of transport
for mezzo in mezzi:
    df[mezzo] = df["che_mezzi"].str.contains(mezzo)

# melt the DataFrame to convert the new columns to a "mezzi" column and a "value" column
melted_df = pd.melt(df, id_vars=["dispositivo"], value_vars=mezzi, var_name="mezzi", value_name="value")

# group the melted DataFrame by "mezzi" and "usa_mezzi" and count the number of users who use each mean of transport
counts_df = melted_df.groupby(["mezzi", "dispositivo"])["value"].sum().reset_index(name="count")

# create a barplot to display the results
sns.histplot(data=counts_df, x="mezzi", weights="count", stat="count", shrink=0.8, palette="summer", hue="dispositivo", multiple="stack", discrete=True)

# set the title and the labels
plt.title("Number of users who use each mean of transport by device")
plt.xlabel("Means of transport")
plt.ylabel("Number of users")
plt.xticks(rotation=45)
plt.show()


Passiamo a un dato importante: i disservizi (rilevanti) più segnalati:

In [ ]:
# disservizi contains a list of disservizi separated by a comma, collect all the unique disservizi
# filter only the ones that appear more than twice

disservizi = df["disservizi"].str.split(",", expand=True).stack().str.strip().value_counts()

disservizi = disservizi[disservizi > 1]

disservizi = disservizi.reset_index()
disservizi.columns = ["disservizio", "numero di segnalazioni"]

g = sns.barplot(x="disservizio", y="numero di segnalazioni", data=disservizi, palette="summer")

g.set_xticklabels(rotation=30, labels=g.get_xticklabels())
g.set(xlabel='Disservizio', ylabel='Numero di segnalazioni')
g.set_title("Disservizi più segnalati")
g.figure.savefig('disservizi.png', dpi=300, bbox_inches='tight')

plt.show()

adesso, per ogni provincia, diamo un occhiata ai mezzi pubblici più usati

In [ ]:
# create a new column for the province of each user
df["province"] = df["provincia_mezzi"].apply(lambda x: x if x in ["Roma", "Milano", "Napoli"] else "Altro")

# split the "che_mezzi" column and create a new column for each mean of transport
for mezzo in mezzi:
    df[mezzo] = df["che_mezzi"].str.contains(mezzo)

# melt the DataFrame to convert the new columns to a "mezzi" column and a "value" column
melted_df = pd.melt(df, id_vars=["province"], value_vars=mezzi, var_name="Mezzi", value_name="value")

# group the melted DataFrame by "province" and "Mezzi" and count the number of users who use each mean of transport
counts_df = melted_df.groupby(["province", "Mezzi"])["value"].sum().reset_index(name="count")

# create a barplot to display the results
sns.barplot(data=counts_df, x="province", y="count", hue="Mezzi", palette="summer", dodge=True, hue_order=["Treno", "Bus", "Tram", "Metro"], order=["Roma", "Milano", "Napoli", "Altro"])

# set the order of the xticks

# set the title and the labels
plt.title("Numero di utenti che utilizzano i mezzi pubblici per provincia")
plt.xlabel("Provincia")
plt.ylabel("Numero di utenti")
plt.xticks(rotation=45)
plt.show()


Un grafico completo (ma piuttosto impegnativo) che incrocia le varie preferenze in 
base all età:

In [ ]:

# define the list of metrics
metrics = ["interesse_crowdsource", "interesse_partecipazione", "interesse_classifica", "interesse_sblocco_feature", "interesse_sblocco_custom", "interesse_donazioni"]




human_readable_metrics = ["Interesse crowdsource", "Intenzione partecipazione", "Classifica", "Sblocco features", "Sblocco personalizzazioni", "Beneficenza"]

mapping = dict(zip(metrics, human_readable_metrics))


# melt the DataFrame to tidy it up
tidy_df = df.melt(id_vars="eta", value_vars=metrics, var_name="metric", value_name="score")

# create a FacetGrid to display the results
g = sns.FacetGrid(tidy_df, col="metric", row="eta", margin_titles=True, sharex=False, height=2, aspect=1.0, row_order=["<18", "18-25", "25-35", "35-55", "55>"],
                  dropna=True, col_order=metrics, despine=True, hue="metric", palette="summer", hue_order=metrics)

# set the title and the labels for each subplot

g.set_titles(col_template="{col_name}", row_template="{row_name}")


#set the xticks for each subplot
g.set(xticks=[0.0, 1.0, 2.0, 3.0, 4.0, 5.0])

# plot the results using a histogram
g.map(sns.histplot, "score", bins=range(0, 6), alpha=0.5, stat="density", kde=True)

# add a legend to the plot
g.add_legend()

# display the plot

g.fig.suptitle("Interesse per le funzionalità dell'app per età")
g.fig.subplots_adjust(top=0.90)


plt.show()

Visto che sono il gruppo più significativo, concentriamoci su i giovani...

In [ ]:
# plot the five metrics only for 18-25 years old users

# create a FacetGrid to display the results

g = sns.FacetGrid(tidy_df[tidy_df["eta"] == "18-25"], col="metric", margin_titles=True, sharex=False, height=2, aspect=1.0)

# set the title and the labels for each subplot

g.set_titles(col_template="{col_name}")

g.set_axis_labels("Score", "")

#set the xticks for each subplot

g.set(xticks=[0.0, 1.0, 2.0, 3.0, 4.0, 5.0])

# plot the results using a histogram

g.map(sns.histplot, "score", bins=range(0, 6), alpha=0.5, stat="density", kde=True, color = "green")

# add a legend to the plot

g.add_legend()

# suptitle

g.fig.suptitle("Interesse per le funzionalità dell'app per età 18-25")
g.fig.subplots_adjust(top=0.75)

plt.show()



Semplifichiamo e, per ogni metrica, mostriamo le preferenze delle varie età una accanto all altra

In [ ]:
# show for each metric a boxplot with the distribution of the scores for each age group

# create a FacetGrid to display the results

g = sns.FacetGrid(tidy_df, col="metric", margin_titles=True, sharex=False, height=2.5, aspect=0.8,
                  despine=True)

# set the title and the labels for each subplot

g.set_titles(col_template="{col_name}")

g.set_axis_labels("", "Score")

# plot the results using a violinplot
g.map(sns.violinplot, "eta", "score", order=["<18", "18-25", "25-35", "35-55", "55>"], palette="summer")

# rotate the xticks
for ax in g.axes.flat:
    
        
    ax.set_xticklabels(ax.get_xticklabels(), rotation=45, horizontalalignment='right')
    

# display the plot
g.figure.subplots_adjust(top=0.75)
g.fig.suptitle("Distribuzione dei punteggi per età")

g.add_legend()

# show

plt.show()

Semplifichiamo ulteriormente: ignoriamo i gruppi di età

In [ ]:
# create a FacetGrid to display the results
g = sns.FacetGrid(tidy_df, col="metric", margin_titles=True, sharex=False, height=2.5, aspect=0.8, despine=True)

# set the title and the labels for each subplot
g.set_titles(col_template="{col_name}")
g.set_axis_labels("", "Density")

# plot the results using a histplot with kde
g.map(sns.histplot, "score", kde=True, bins=range(0, 6), stat="density", alpha=0.5, color="green")

# display the plot
g.fig.suptitle("Distribution of Scores for each Metric")
g.fig.subplots_adjust(top=0.85)

# set the xticks for each subplot

g.set(xticks=[0.0, 1.0, 2.0, 3.0, 4.0, 5.0])

# show
plt.show()


In conclusione, diamo uno sguardo alle feature *approvate*, ovvero quelle che hanno ricevuto una 
maggioranza di voti $\geq 3$:

In [ ]:
# For each metric, say if the majority of users are interested in it or not

tidy_df["interested"] = tidy_df["score"] >= 3

tidy_df["interested"] = tidy_df["interested"].replace({True: "Interested", False: "Not interested"})

# count the number of users for each metric and for each category

counts_df = tidy_df.groupby(["metric", "interested"]).size().reset_index(name="count")

# plot the results using a barplot

g = sns.barplot(x="metric", y="count", hue="interested", data=counts_df, hue_order=["Interested", "Not interested"])

plt.title("Numero di utenti interessati per metrica")

g.set_xticklabels(g.get_xticklabels(), rotation=30, horizontalalignment='right')

plt.show()